![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Training and Reusing Clinical Named Entity Recognition Models

Please make sure that your cluster is setup properly according to https://nlp.johnsnowlabs.com/docs/en/licensed_install#install-spark-nlp-for-healthcare-on-databricks

## Blogposts and videos:

https://towardsdatascience.com/named-entity-recognition-ner-with-bert-in-spark-nlp-874df20d1d77

https://www.youtube.com/watch?v=YM-e4eOiQ34

https://medium.com/spark-nlp/named-entity-recognition-for-healthcare-with-sparknlp-nerdl-and-nercrf-a7751b6ad571

https://medium.com/atlas-research/ner-for-clinical-text-7c73caddd180

In [0]:
import os
import json
import string
import numpy as np
import pandas as pd


import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.util import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.pretrained import ResourceDownloader

from pyspark.sql import functions as F
from pyspark.ml import Pipeline, PipelineModel

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)


print('sparknlp.version : ',sparknlp.version())
print('sparknlp_jsl.version : ',sparknlp_jsl.version())

spark

sparknlp.version : 3.1.0
sparknlp_jsl.version : 3.1.1
Out[20]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.1.1 
 Master 
 spark://10.139.64.4:7077 
 AppName 
 Databricks Shell

# Clinical NER Pipeline (with pretrained models)

In [0]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP
documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line
sentenceDetector = SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare","en","clinical/models") \
  .setInputCols(["document"]) \
  .setOutputCol("sentence") 

#sentenceDetector = SentenceDetector()\
#   .setInputCols(["document"])\
#   .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("embeddings")

# NER model trained on i2b2 (sampled from MIMIC) dataset
clinical_ner = MedicalNerModel.pretrained("ner_clinical_large", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("ner")

ner_converter = NerConverter() \
  .setInputCols(["sentence", "token", "ner"]) \
  .setOutputCol("ner_chunk")

nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    clinical_ner,
    ner_converter])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 363.9 KB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]
ner_clinical_large download started this may take some time.
Approximate size to download 13.9 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]

In [0]:
#checking the stages in pipeline
model.stages

Out[3]: [DocumentAssembler_c8475b6c16f0,
 SentenceDetectorDLModel_d2546f0acfe2,
 REGEX_TOKENIZER_77bb96848c34,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 MedicalNerModel_1a8637089929,
 NerConverter_6d7c90140772]

In [0]:
#getting the classes in pretrained model
clinical_ner.getClasses()

Out[4]: ['O',
 'B-TREATMENT',
 'I-TREATMENT',
 'B-PROBLEM',
 'I-PROBLEM',
 'B-TEST',
 'I-TEST']

In [0]:
#extracting the embedded default param values
clinical_ner.extractParamMap()

Out[5]: {Param(parent='MedicalNerModel_1a8637089929', name='includeAllConfidenceScores', doc='whether to include all confidence scores in annotation metadata or just the score of the predicted tag'): False,
 Param(parent='MedicalNerModel_1a8637089929', name='inferenceBatchSize', doc='number of sentences to process in a single batch during inference'): 1,
 Param(parent='MedicalNerModel_1a8637089929', name='lazyAnnotator', doc='Whether this AnnotatorModel acts as lazy in RecursivePipelines'): False,
 Param(parent='MedicalNerModel_1a8637089929', name='includeConfidence', doc='whether to include confidence scores in annotation metadata'): True,
 Param(parent='MedicalNerModel_1a8637089929', name='batchSize', doc='Size of every batch'): 64,
 Param(parent='MedicalNerModel_1a8637089929', name='classes', doc='get the tags used to trained this MedicalNerModel'): ['O',
 'B-TREATMENT',
 'I-TREATMENT',
 'B-PROBLEM',
 'I-PROBLEM',
 'B-TEST',
 'I-TEST'],
 Param(parent='MedicalNerModel_1a8637089929', name='inputCols', doc='previous annotations columns, if renamed'): ['sentence',
 'token',
 'embeddings'],
 Param(parent='MedicalNerModel_1a8637089929', name='outputCol', doc='output annotation column. can be left default.'): 'ner',
 Param(parent='MedicalNerModel_1a8637089929', name='storageRef', doc='unique reference name for identification'): 'clinical'}

In [0]:
#checking the embeddings
clinical_ner.getStorageRef()

Out[6]: 'clinical'

In [0]:
#downloading the sample dataset
! wget -q https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/pubmed/pubmed_sample_text_small.csv

dbutils.fs.cp("file:/databricks/driver/petfinder-mini.csv", "dbfs:/") 

Out[7]: True

In [0]:
%fs ls file:/databricks/driver 

path,name,size
file:/databricks/driver/conf/,conf/,4096
file:/databricks/driver/preload_class.lst,preload_class.lst,750334
file:/databricks/driver/logs/,logs/,4096
file:/databricks/driver/dbfs:/,dbfs:/,4096
file:/databricks/driver/pubmed_sample_text_small.csv,pubmed_sample_text_small.csv,9363435
file:/databricks/driver/sample-sentences-en.txt,sample-sentences-en.txt,284
file:/databricks/driver/regex_rules.txt,regex_rules.txt,195
file:/databricks/driver/annotators.csv,annotators.csv,3136
file:/databricks/driver/AntBNC_lemmas_ver_001.txt,AntBNC_lemmas_ver_001.txt,1348552
file:/databricks/driver/pubmed-sample.csv,pubmed-sample.csv,10484510


In [0]:
import pyspark.sql.functions as F

pubMedDF = spark.read\
                .option("header", "true")\
                .csv("dbfs:/pubmed_sample_text_small.csv")\
                
pubMedDF.show(truncate=80)

+--------------------------------------------------------------------------------+
 text|
+--------------------------------------------------------------------------------+
The human KCNJ9 (Kir 3.3, GIRK3) is a member of the G-protein-activated inwar...|
BACKGROUND: At present, it is one of the most important issues for the treatm...|
OBJECTIVE: To investigate the relationship between preoperative atrialfibrill...|
Combined EEG/fMRI recording has been used to localize the generators of EEGev...|
Kohlschutter syndrome is a rare neurodegenerative disorder presenting withint...|
Statistical analysis of neuroimages is commonly approached with intergroupcom...|
The synthetic DOX-LNA conjugate was characterized by proton nuclear magneticr...|
Our objective was to compare three different methods of blood pressuremeasure...|
We conducted a phase II study to assess the efficacy and tolerability ofirino...|
"""Monomeric sarcosine oxidase (MSOX) is a flavoenzyme that catalyzes the oxi...|
We presented the tachinid fly Exorista japonica with moving host models: afre...|
The literature dealing with the water conducting properties of sapwood xylem ...|
A novel approach to synthesize chitosan-O-isopropyl-5'-O-d4T monophosphatecon...|
An HPLC-ESI-MS-MS method has been developed for the quantitative determinatio...|
The localizing and lateralizing values of eye and head ictal deviations durin...|
OBJECTIVE: To evaluate the effectiveness and acceptability of expectantmanage...|
For the construction of new combinatorial libraries, a lead compound was crea...|
We report the results of a screen for genetic association with urinary arseni...|
Intraparenchymal pericatheter cyst is rarely reported. Obstruction in thevent...|
It is known that patients with Klinefelter's syndrome are inclined to develop...|
+--------------------------------------------------------------------------------+
only showing top 20 rows

In [0]:
pubMedDF.printSchema()

root
-- text: string (nullable = true)

In [0]:
result = model.transform(pubMedDF.limit(100))

In [0]:
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| ner| ner_chunk|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
The human KCNJ9 (...|[{document, 0, 95...|[{document, 0, 12...|[{token, 0, 2, Th...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 48, 106,...|
BACKGROUND: At pr...|[{document, 0, 14...|[{document, 0, 19...|[{token, 0, 9, BA...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 67, 79, ...|
OBJECTIVE: To inv...|[{document, 0, 15...|[{document, 0, 14...|[{token, 0, 8, OB...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 130, 144...|
Combined EEG/fMRI...|[{document, 0, 16...|[{document, 0, 29...|[{token, 0, 7, Co...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 26, C...|
Kohlschutter synd...|[{document, 0, 25...|[{document, 0, 20...|[{token, 0, 11, K...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 20, K...|
Statistical analy...|[{document, 0, 10...|[{document, 0, 22...|[{token, 0, 10, S...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 34, S...|
The synthetic DOX...|[{document, 0, 57...|[{document, 0, 10...|[{token, 0, 2, Th...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 30, T...|
Our objective was...|[{document, 0, 24...|[{document, 0, 19...|[{token, 0, 2, Ou...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 56, 80, ...|
We conducted a ph...|[{document, 0, 14...|[{document, 0, 24...|[{token, 0, 1, We...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 13, 28, ...|
"""Monomeric sarc...|[{document, 0, 14...|[{document, 0, 14...|[{token, 0, 2, ""...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 3, 36, M...|
We presented the ...|[{document, 0, 12...|[{document, 0, 34...|[{token, 0, 1, We...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 13, 46, ...|
The literature de...|[{document, 0, 16...|[{document, 0, 13...|[{token, 0, 2, Th...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 154, 166...|
A novel approach ...|[{document, 0, 64...|[{document, 0, 97...|[{token, 0, 0, A,...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 99, 124,...|
An HPLC-ESI-MS-MS...|[{document, 0, 90...|[{document, 0, 24...|[{token, 0, 1, An...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 23, A...|
The localizing an...|[{document, 0, 72...|[{document, 0, 12...|[{token, 0, 2, Th...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 50, 70, ...|
OBJECTIVE: To eva...|[{document, 0, 13...|[{document, 0, 14...|[{token, 0, 8, OB...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 97, 143,...|
For the construct...|[{document, 0, 32...|[{document, 0, 20...|[{token, 0, 2, Fo...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 24, 50, ...|
We report the res...|[{document, 0, 13...|[{document, 0, 31...|[{token, 0, 1, We...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 25, 32, ...|
Intraparenchymal ...|[{document, 0, 10...|[{document, 0, 53...|[{token, 0, 15, I...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 33, I...|
It is known that ...|[{document, 0, 34...|[{document, 0, 14...|[{token, 0, 1, It...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 31, 52, ...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [0]:
result.select('token.result','ner.result').show(truncate=80)

+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
 result| result|
+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
[The, human, KCNJ9, (, Kir, 3.3, ,, GIRK3, ), is, a, member, of, the, G-prote...|[O, O, O, O, O, O, O, O, O, O, O, O, O, B-TREATMENT, I-TREATMENT, I-TREATMENT...|
[BACKGROUND, :, At, present, ,, it, is, one, of, the, most, important, issues...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-TREATMENT, I-TREATMENT, O, B-PRO...|
[OBJECTIVE, :, To, investigate, the, relationship, between, preoperative, atr...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-TREATMENT, I-TREATMENT, O,...|
[Combined, EEG/fMRI, recording, has, been, used, to, localize, the, generator...|[B-TEST, I-TEST, I-TEST, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-TEST,...|
[Kohlschutter, syndrome, is, a, rare, neurodegenerative, disorder, presenting...|[B-PROBLEM, I-PROBLEM, O, B-PROBLEM, I-PROBLEM, I-PROBLEM, I-PROBLEM, O, B-PR...|
[Statistical, analysis, of, neuroimages, is, commonly, approached, with, inte...|[B-TEST, I-TEST, I-TEST, I-TEST, O, O, O, O, B-TREATMENT, O, O, O, O, O, O, O...|
[The, synthetic, DOX-LNA, conjugate, was, characterized, by, proton, nuclear,...|[B-PROBLEM, I-PROBLEM, I-PROBLEM, I-PROBLEM, O, O, O, B-PROBLEM, I-PROBLEM, I...|
[Our, objective, was, to, compare, three, different, methods, of, blood, pres...|[O, O, O, O, O, O, O, O, O, B-TREATMENT, I-TREATMENT, O, O, O, O, B-TEST, I-T...|
[We, conducted, a, phase, II, study, to, assess, the, efficacy, and, tolerabi...|[O, O, B-TEST, I-TEST, I-TEST, I-TEST, O, O, B-TREATMENT, I-TREATMENT, O, B-T...|
[""", Monomeric, sarcosine, oxidase, (, MSOX, ), is, a, flavoenzyme, that, ca...|[O, B-TREATMENT, I-TREATMENT, I-TREATMENT, I-TREATMENT, I-TREATMENT, I-TREATM...|
[We, presented, the, tachinid, fly, Exorista, japonica, with, moving, host, m...|[O, O, B-PROBLEM, I-PROBLEM, I-PROBLEM, I-PROBLEM, I-PROBLEM, O, O, O, O, O, ...|
[The, literature, dealing, with, the, water, conducting, properties, of, sapw...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-PR...|
[A, novel, approach, to, synthesize, chitosan-O-isopropyl-5'-O-d4T, monophosp...|[O, O, O, O, O, O, O, O, O, O, B-TREATMENT, I-TREATMENT, O, O, B-PROBLEM, I-P...|
[An, HPLC-ESI-MS-MS, method, has, been, developed, for, the, quantitative, de...|[B-TEST, I-TEST, I-TEST, O, O, O, O, O, O, O, B-PROBLEM, I-PROBLEM, O, B-TREA...|
[The, localizing, and, lateralizing, values, of, eye, and, head, ictal, devia...|[O, O, O, O, O, O, O, O, B-PROBLEM, I-PROBLEM, I-PROBLEM, B-PROBLEM, I-PROBLE...|
[OBJECTIVE, :, To, evaluate, the, effectiveness, and, acceptability, of, expe...|[O, O, O, O, O, O, O, O, O, O, O, O, O, B-PROBLEM, I-PROBLEM, I-PROBLEM, I-PR...|
[For, the, construction, of, new, combinatorial, libraries, ,, a, lead, compo...|[O, O, O, O, B-PROBLEM, I-PROBLEM, I-PROBLEM, O, B-PROBLEM, I-PROBLEM, I-PROB...|
[We, report, the, results, of, a, screen, for, genetic, association, with, ur...|[O, O, O, O, O, B-TEST, I-TEST, O, O, O, O, B-PROBLEM, I-PROBLEM, I-PROBLEM, ...|
[Intraparenchymal, pericatheter, cyst, is, rarely, reported, ., Obstruction, ...|[B-PROBLEM, I-PROBLEM, I-PROBLEM, O, O, O, O, B-PROBLEM, I-PROBLEM, I-PROBLEM...|
[It, is, known, that, patients, with, Klinefelter's, syndrome, are, inclined,...|[O, O, O, O, O, O, B-PROBLEM, I-PROBLEM, O, O, O, B-PROBLEM, I-PROBLEM, I-PRO...|
+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
only showing top 20 rows

In [0]:
result_df = result.select(F.explode(F.arrays_zip('token.result', 'ner.result')).alias("cols")) \
                  .select(F.expr("cols['0']").alias("token"),
                          F.expr("cols['1']").alias("ner_label"))

result_df.show(50, truncate=100)

+-------------------+-----------+
 token| ner_label|
+-------------------+-----------+
 The| O|
 human| O|
 KCNJ9| O|
 (| O|
 Kir| O|
 3.3| O|
 ,| O|
 GIRK3| O|
 )| O|
 is| O|
 a| O|
 member| O|
 of| O|
 the|B-TREATMENT|
G-protein-activated|I-TREATMENT|
 inwardly|I-TREATMENT|
 rectifying|I-TREATMENT|
 potassium|I-TREATMENT|
 (|I-TREATMENT|
 GIRK|I-TREATMENT|
 )| O|
 channel| O|
 family| O|
 .| O|
 Here| O|
 we| O|
 describe| O|
 the|B-TREATMENT|
genomicorganization|I-TREATMENT|
 of| O|
 the| O|
 KCNJ9| O|
 locus| O|
 on| O|
 chromosome| O|
 1q21-23| O|
 as| O|
 a| B-PROBLEM|
 candidate| I-PROBLEM|
 gene| I-PROBLEM|
 forType| I-PROBLEM|
 II| I-PROBLEM|
 diabetes| I-PROBLEM|
 mellitus| I-PROBLEM|
 in| O|
 the| O|
 Pima| O|
 Indian| O|
 population| O|
 .| O|
+-------------------+-----------+
only showing top 50 rows

In [0]:
result_df.select("token", "ner_label").groupBy('ner_label').count().orderBy('count', ascending=False).show(truncate=False)

+-----------+-----+
ner_label |count|
+-----------+-----+
O |13874|
I-PROBLEM |1594 |
B-PROBLEM |915 |
I-TREATMENT|737 |
I-TEST |634 |
B-TREATMENT|623 |
B-TEST |534 |
+-----------+-----+

In [0]:
result.select('ner_chunk').take(1)

Out[15]: [Row(ner_chunk=[Row(annotatorType='chunk', begin=48, end=106, result='the G-protein-activated inwardly rectifying potassium (GIRK', metadata={'sentence': '0', 'chunk': '0', 'entity': 'TREATMENT'}, embeddings=[]), Row(annotatorType='chunk', begin=142, end=164, result='the genomicorganization', metadata={'sentence': '1', 'chunk': '1', 'entity': 'TREATMENT'}, embeddings=[]), Row(annotatorType='chunk', begin=210, end=254, result='a candidate gene forType II diabetes mellitus', metadata={'sentence': '1', 'chunk': '2', 'entity': 'PROBLEM'}, embeddings=[]), Row(annotatorType='chunk', begin=380, end=394, result='byapproximately', metadata={'sentence': '2', 'chunk': '3', 'entity': 'TREATMENT'}, embeddings=[]), Row(annotatorType='chunk', begin=464, end=494, result='single nucleotide polymorphisms', metadata={'sentence': '3', 'chunk': '4', 'entity': 'TREATMENT'}, embeddings=[]), Row(annotatorType='chunk', begin=532, end=554, result='aVal366Ala substitution', metadata={'sentence': '3', 'chunk': '5', 'entity': 'PROBLEM'}, embeddings=[]), Row(annotatorType='chunk', begin=561, end=574, result='an 8 base-pair', metadata={'sentence': '3', 'chunk': '6', 'entity': 'PROBLEM'}, embeddings=[]), Row(annotatorType='chunk', begin=581, end=598, result='insertion/deletion', metadata={'sentence': '3', 'chunk': '7', 'entity': 'PROBLEM'}, embeddings=[]), Row(annotatorType='chunk', begin=601, end=621, result='Ourexpression studies', metadata={'sentence': '4', 'chunk': '8', 'entity': 'TEST'}, embeddings=[]), Row(annotatorType='chunk', begin=648, end=685, result='the transcript in various humantissues', metadata={'sentence': '4', 'chunk': '9', 'entity': 'PROBLEM'}, embeddings=[]), Row(annotatorType='chunk', begin=749, end=770, result='fat andskeletal muscle', metadata={'sentence': '4', 'chunk': '10', 'entity': 'PROBLEM'}, embeddings=[]), Row(annotatorType='chunk', begin=830, end=843, result='furtherstudies', metadata={'sentence': '5', 'chunk': '11', 'entity': 'PROBLEM'}, embeddings=[]), Row(annotatorType='chunk', begin=864, end=880, result='the KCNJ9 protein', metadata={'sentence': '5', 'chunk': '12', 'entity': 'TREATMENT'}, embeddings=[]), Row(annotatorType='chunk', begin=892, end=901, result='evaluation', metadata={'sentence': '5', 'chunk': '13', 'entity': 'TEST'}, embeddings=[]), Row(annotatorType='chunk', begin=940, end=955, result='Type II diabetes', metadata={'sentence': '5', 'chunk': '14', 'entity': 'PROBLEM'}, embeddings=[])])]

In [0]:
result.select(F.explode(F.arrays_zip('ner_chunk.result', 'ner_chunk.metadata')).alias("cols")) \
.select(F.expr("cols['0']").alias("chunk"),
        F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+-----------------------------------------------------------+---------+
chunk |ner_label|
+-----------------------------------------------------------+---------+
the G-protein-activated inwardly rectifying potassium (GIRK|TREATMENT|
the genomicorganization |TREATMENT|
a candidate gene forType II diabetes mellitus |PROBLEM |
byapproximately |TREATMENT|
single nucleotide polymorphisms |TREATMENT|
aVal366Ala substitution |PROBLEM |
an 8 base-pair |PROBLEM |
insertion/deletion |PROBLEM |
Ourexpression studies |TEST |
the transcript in various humantissues |PROBLEM |
fat andskeletal muscle |PROBLEM |
furtherstudies |PROBLEM |
the KCNJ9 protein |TREATMENT|
evaluation |TEST |
Type II diabetes |PROBLEM |
the treatment |TREATMENT|
breast cancer |PROBLEM |
the standard therapy |TREATMENT|
anthracyclines |TREATMENT|
taxanes |TREATMENT|
+-----------------------------------------------------------+---------+
only showing top 20 rows

In [0]:
# fullAnnotate in LightPipeline

text = '''
A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting . Two weeks prior to presentation , she was treated with a five-day course of amoxicillin for a respiratory tract infection . She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . She had been on dapagliflozin for six months at the time of presentation . Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness , guarding , or rigidity . Pertinent laboratory findings on admission were : serum glucose 111 mg/dl , bicarbonate 18 mmol/l , anion gap 20 , creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , glycated hemoglobin ( HbA1c ) 10% , and venous pH 7.27 . Serum lipase was normal at 43 U/L . Serum acetone levels could not be assessed as blood samples kept hemolyzing due to significant lipemia . The patient was initially admitted for starvation ketosis , as she reported poor oral intake for three days prior to admission . However , serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL , the anion gap was still elevated at 21 , serum bicarbonate was 16 mmol/L , triglyceride level peaked at 2050 mg/dL , and lipase was 52 U/L . The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again . The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL , within 24 hours . Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use . The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely . She had close follow-up with endocrinology post discharge .
'''

print (text)

light_model = LightPipeline(model)

light_result = light_model.fullAnnotate(text)


chunks = []
entities = []
sentence= []
begin = []
end = []

for n in light_result[0]['ner_chunk']:
        
    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    sentence.append(n.metadata['sentence'])
    
    
import pandas as pd

df = pd.DataFrame({'chunks':chunks, 'begin': begin, 'end':end, 
                   'sentence_id':sentence, 'entities':entities})

df.head(20)

A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting . Two weeks prior to presentation , she was treated with a five-day course of amoxicillin for a respiratory tract infection . She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . She had been on dapagliflozin for six months at the time of presentation . Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness , guarding , or rigidity . Pertinent laboratory findings on admission were : serum glucose 111 mg/dl , bicarbonate 18 mmol/l , anion gap 20 , creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , glycated hemoglobin ( HbA1c ) 10% , and venous pH 7.27 . Serum lipase was normal at 43 U/L . Serum acetone levels could not be assessed as blood samples kept hemolyzing due to significant lipemia . The patient was initially admitted for starvation ketosis , as she reported poor oral intake for three days prior to admission . However , serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL , the anion gap was still elevated at 21 , serum bicarbonate was 16 mmol/L , triglyceride level peaked at 2050 mg/dL , and lipase was 52 U/L . The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again . The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL , within 24 hours . Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use . The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely . She had close follow-up with endocrinology post discharge .

Out[17]:

,chunks,begin,end,sentence_id,entities
0,gestational diabetes mellitus,40,68,0,PROBLEM
1,subsequent type two diabetes mellitus,118,154,0,PROBLEM
2,T2DM,158,161,0,PROBLEM
3,HTG-induced pancreatitis,187,210,0,PROBLEM
4,an acute hepatitis,264,281,0,PROBLEM
5,obesity,289,295,0,PROBLEM
6,a body mass index,302,318,0,PROBLEM
7,BMI,322,324,0,TEST
8,polyuria,381,388,0,PROBLEM
9,polydipsia,392,401,0,PROBLEM


In [0]:
from sparknlp_display import NerVisualizer

visualiser = NerVisualizer()

ner_vis = visualiser.display(light_result[0], label_col='ner_chunk', document_col='document', return_html=True)

# Change color of an entity label

#visualiser.set_label_colors({'PROBLEM':'#008080', 'TEST':'#800080', 'TREATMENT':'#808080'})
#visualiser.display(light_result[0], label_col='ner_chunk')

# Set label filter

# visualiser.display(light_result, label_col='ner_chunk', document_col='document',
                   #labels=['PROBLEM','TEST'])
  
displayHTML(ner_vis)

A 28-year-old female with a history of gestational diabetes mellitus PROBLEM diagnosed eight years prior to presentation and subsequent type two diabetes mellitus PROBLEM ( T2DM PROBLEM ), one prior episode of HTG-induced pancreatitis PROBLEM three years prior to presentation , associated with an acute hepatitis PROBLEM , and obesity PROBLEM with a body mass index PROBLEM ( BMI TEST ) of 33.5 kg/m2 , presented with a one-week history of polyuria PROBLEM , polydipsia PROBLEM , poor appetite PROBLEM , and vomiting PROBLEM . Two weeks prior to presentation , she was treated with a five-day course of amoxicillin TREATMENT for a respiratory tract infection PROBLEM . She was on metformin TREATMENT , glipizide TREATMENT , and dapagliflozin TREATMENT for T2DM PROBLEM and atorvastatin TREATMENT and gemfibrozil TREATMENT for HTG PROBLEM . She had been on dapagliflozin TREATMENT for six months at the time of presentation . Physical examination TEST on presentation was significant for dry oral mucosa PROBLEM ; significantly , her abdominal examination TEST was benign with no tenderness PROBLEM , guarding PROBLEM , or rigidity PROBLEM . Pertinent laboratory findings on admission were : serum glucose TEST 111 mg/dl , bicarbonate TEST 18 mmol/l , anion gap TEST 20 , creatinine TEST 0.4 mg/dL , triglycerides TEST 508 mg/dL , total cholesterol TEST 122 mg/dL , glycated hemoglobin TEST ( HbA1c TEST ) 10% , and venous pH TEST 7.27 . Serum lipase TEST was normal at 43 U/L TEST . Serum acetone levels TEST could not be assessed as blood samples TEST kept hemolyzing PROBLEM due to significant lipemia PROBLEM . The patient was initially admitted for starvation ketosis PROBLEM , as she reported poor oral intake PROBLEM for three days prior to admission . However , serum chemistry TEST obtained six hours after presentation revealed her glucose TEST was 186 mg/dL , the anion gap TEST was still elevated PROBLEM at 21 , serum bicarbonate TEST was 16 mmol/L , triglyceride level TEST peaked at 2050 mg/dL , and lipase TEST was 52 U/L TEST . The β-hydroxybutyrate level TEST was obtained and found to be elevated PROBLEM at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis TEST due to interference from turbidity PROBLEM caused by lipemia PROBLEM again . The patient was treated with an insulin drip TREATMENT for euDKA PROBLEM and HTG PROBLEM with a reduction TREATMENT in the anion gap TEST to 13 and triglycerides TEST to 1400 mg/dL , within 24 hours . Her euDKA was thought to be precipitated by her respiratory tract infection PROBLEM in the setting of SGLT2 inhibitor TREATMENT use . The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine TREATMENT at night , 12 units of insulin lispro TREATMENT with meals , and metformin TREATMENT 1000 mg two times a day . It was determined that all SGLT2 inhibitors TREATMENT should be discontinued indefinitely . She had close follow-up with endocrinology post discharge .

## NER JSL Model

Let's show an example of `ner_jsl` model that has about 80 labels by changing just only the model name.

In [0]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP
documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line
sentenceDetector = SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare","en","clinical/models") \
  .setInputCols(["document"]) \
  .setOutputCol("sentence") 

#sentenceDetector = SentenceDetector()\
#   .setInputCols(["document"])\
#   .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("embeddings")

# NER model trained on i2b2 (sampled from MIMIC) dataset
jsl_ner = MedicalNerModel.pretrained("ner_jsl", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("jsl_ner")

jsl_ner_converter = NerConverter() \
  .setInputCols(["sentence", "token", "jsl_ner"]) \
  .setOutputCol("jsl_ner_chunk")

jsl_ner_pipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    jsl_ner,
    jsl_ner_converter])

empty_data = spark.createDataFrame([[""]]).toDF("text")

jsl_ner_model = jsl_ner_pipeline.fit(empty_data)

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 363.9 KB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_jsl download started this may take some time.
Approximate size to download 14.5 MB
[ | ][ / ][ — ][ \ ][OK!]

In [0]:
print (text)

jsl_light_model = LightPipeline(jsl_ner_model)

jsl_light_result = jsl_light_model.fullAnnotate(text)


chunks = []
entities = []
sentence= []
begin = []
end = []

for n in jsl_light_result[0]['jsl_ner_chunk']:
        
    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    sentence.append(n.metadata['sentence'])
    
    
import pandas as pd

df = pd.DataFrame({'chunks':chunks, 'begin': begin, 'end':end, 
                   'sentence_id':sentence, 'entities':entities})

df.head(20)

A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting . Two weeks prior to presentation , she was treated with a five-day course of amoxicillin for a respiratory tract infection . She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . She had been on dapagliflozin for six months at the time of presentation . Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness , guarding , or rigidity . Pertinent laboratory findings on admission were : serum glucose 111 mg/dl , bicarbonate 18 mmol/l , anion gap 20 , creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , glycated hemoglobin ( HbA1c ) 10% , and venous pH 7.27 . Serum lipase was normal at 43 U/L . Serum acetone levels could not be assessed as blood samples kept hemolyzing due to significant lipemia . The patient was initially admitted for starvation ketosis , as she reported poor oral intake for three days prior to admission . However , serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL , the anion gap was still elevated at 21 , serum bicarbonate was 16 mmol/L , triglyceride level peaked at 2050 mg/dL , and lipase was 52 U/L . The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again . The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL , within 24 hours . Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use . The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely . She had close follow-up with endocrinology post discharge .

Out[26]:

,chunks,begin,end,sentence_id,entities
0,28-year-old,3,13,0,Age
1,female,15,20,0,Gender
2,gestational diabetes mellitus,40,68,0,Diabetes
3,eight years prior,80,96,0,RelativeDate
4,subsequent,118,127,0,Modifier
5,type two diabetes mellitus,129,154,0,Diabetes
6,T2DM,158,161,0,Diabetes
7,HTG-induced pancreatitis,187,210,0,Disease_Syndrome_Disorder
8,three years prior,212,228,0,RelativeDate
9,acute,267,271,0,Modifier


In [0]:
from sparknlp_display import NerVisualizer

visualiser = NerVisualizer()

ner_vis = visualiser.display(jsl_light_result[0], label_col='jsl_ner_chunk', document_col='document', return_html=True)

displayHTML(ner_vis)

A 28-year-old Age female Gender with a history of gestational diabetes mellitus Diabetes diagnosed eight years prior RelativeDate to presentation and subsequent Modifier type two diabetes mellitus Diabetes ( T2DM Diabetes ), one prior episode of HTG-induced pancreatitis Disease_Syndrome_Disorder three years prior RelativeDate to presentation , associated with an acute Modifier hepatitis Communicable_Disease , and obesity Obesity with a body mass index Symptom ( BMI ) of 33.5 kg/m2 Weight , presented with a one-week Duration history of polyuria Symptom , polydipsia Symptom , poor appetite Symptom , and vomiting Symptom . Two weeks prior RelativeDate to presentation , she Gender was treated with a five-day Frequency course of amoxicillin Drug_Ingredient for a respiratory tract infection Disease_Syndrome_Disorder . She Gender was on metformin Drug_Ingredient , glipizide Drug_Ingredient , and dapagliflozin Drug_Ingredient for T2DM Drug_Ingredient and atorvastatin Drug_Ingredient and gemfibrozil Drug_Ingredient for HTG Hyperlipidemia . She Gender had been on dapagliflozin Drug_Ingredient for six months Duration at the time of presentation . Physical examination on presentation was significant for dry oral mucosa Symptom ; significantly , her Gender abdominal External_body_part_or_region examination was benign with no tenderness Symptom , guarding Symptom , or rigidity Symptom . Pertinent laboratory findings on admission Admission_Discharge were : serum glucose Test 111 mg/dl Test_Result , bicarbonate Test 18 mmol/l Test_Result , anion gap Test 20 Test_Result , creatinine Test 0.4 mg/dL Test_Result , triglycerides 508 mg/dL Triglycerides , total cholesterol Test 122 mg/dL Test_Result , glycated hemoglobin Test ( HbA1c Test ) 10% Test_Result , and venous pH Test 7.27 Test_Result . Serum lipase Test was normal Test_Result at 43 U/L Test_Result . Serum acetone levels Test could not be assessed as blood samples Test kept hemolyzing Test_Result due to significant Modifier lipemia Symptom . The patient was initially admitted Admission_Discharge for starvation ketosis Disease_Syndrome_Disorder , as she Gender reported poor oral intake Symptom for three days Duration prior to admission Admission_Discharge . However , serum chemistry Test obtained six hours RelativeTime after presentation revealed her Gender glucose Test was 186 mg/dL Test_Result , the anion gap Test was still elevated Test_Result at 21 Test_Result , serum bicarbonate Test was 16 mmol/L Test_Result , triglyceride level Test peaked at 2050 mg/dL Test_Result , and lipase Test was 52 U/L Test_Result . The β-hydroxybutyrate level Test was obtained and found to be elevated Test_Result at 5.29 mmol/L Test_Result - the original sample was centrifuged and the chylomicron layer Internal_organ_or_component removed prior to analysis due to interference from turbidity Symptom caused by lipemia Symptom again . The patient was treated with an insulin Drug_Ingredient drip Route for euDKA and HTG Drug_Ingredient with a reduction in the anion gap to 13 Test_Result and triglycerides Test to 1400 mg/dL Test_Result , within 24 hours RelativeTime . Her Gender euDKA was thought to be precipitated by her Gender respiratory tract infection Disease_Syndrome_Disorder in the setting of SGLT2 inhibitor Drug_Ingredient use . The patient was seen by the endocrinology service Clinical_Dept and she Gender was discharged Admission_Discharge on 40 units Dosage of insulin glargine Drug_Ingredient at night Frequency , 12 units Dosage of insulin lispro Drug_Ingredient with meals Frequency , and metformin Drug_Ingredient 1000 mg Strength two times a day Frequency . It was determined that all SGLT2 inhibitors Drug_Ingredient should be discontinued indefinitely . She Gender had close follow-up with endocrinology Clinical_Dept post discharge Admission_Discharge .

## Posology NER

In [0]:
# NER model trained on i2b2 (sampled from MIMIC) dataset
posology_ner = MedicalNerModel.pretrained("ner_posology", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")

ner_converter = NerConverter()\
    .setInputCols(["sentence","token","ner"])\
    .setOutputCol("ner_chunk")

posology_ner_greedy = MedicalNerModel.pretrained("ner_posology_greedy", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner_greedy")

ner_converter_greedy = NerConverter()\
    .setInputCols(["sentence","token","ner_greedy"])\
    .setOutputCol("ner_chunk_greedy")

nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    posology_ner,
    ner_converter,
    posology_ner_greedy,
    ner_converter_greedy])

empty_data = spark.createDataFrame([[""]]).toDF("text")
posology_model = nlpPipeline.fit(empty_data)

ner_posology download started this may take some time.
Approximate size to download 13.8 MB
[ | ][ / ][ — ][ \ ][ | ][OK!]
ner_posology_greedy download started this may take some time.
Approximate size to download 13.9 MB
[ | ][ / ][ — ][ \ ][OK!]

In [0]:
posology_ner.getClasses()

Out[29]: ['O',
 'B-DOSAGE',
 'B-STRENGTH',
 'I-STRENGTH',
 'B-ROUTE',
 'B-FREQUENCY',
 'I-FREQUENCY',
 'B-DRUG',
 'I-DRUG',
 'B-FORM',
 'I-DOSAGE',
 'B-DURATION',
 'I-DURATION',
 'I-FORM',
 'I-ROUTE']

In [0]:
posology_result = posology_model.transform(pubMedDF.limit(100))

In [0]:
posology_result.show(10)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| ner| ner_chunk| ner_greedy| ner_chunk_greedy|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
The human KCNJ9 (...|[{document, 0, 95...|[{document, 0, 12...|[{token, 0, 2, Th...|[{word_embeddings...|[{named_entity, 0...| []|[{named_entity, 0...|[{chunk, 52, 122,...|
BACKGROUND: At pr...|[{document, 0, 14...|[{document, 0, 19...|[{token, 0, 9, BA...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 167, 180...|[{named_entity, 0...|[{chunk, 167, 180...|
OBJECTIVE: To inv...|[{document, 0, 15...|[{document, 0, 14...|[{token, 0, 8, OB...|[{word_embeddings...|[{named_entity, 0...| []|[{named_entity, 0...| []|
Combined EEG/fMRI...|[{document, 0, 16...|[{document, 0, 29...|[{token, 0, 7, Co...|[{word_embeddings...|[{named_entity, 0...| []|[{named_entity, 0...| []|
Kohlschutter synd...|[{document, 0, 25...|[{document, 0, 20...|[{token, 0, 11, K...|[{word_embeddings...|[{named_entity, 0...| []|[{named_entity, 0...| []|
Statistical analy...|[{document, 0, 10...|[{document, 0, 22...|[{token, 0, 10, S...|[{word_embeddings...|[{named_entity, 0...| []|[{named_entity, 0...| []|
The synthetic DOX...|[{document, 0, 57...|[{document, 0, 10...|[{token, 0, 2, Th...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 432, 446...|[{named_entity, 0...|[{chunk, 432, 446...|
Our objective was...|[{document, 0, 24...|[{document, 0, 19...|[{token, 0, 2, Ou...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 174, 185...|[{named_entity, 0...|[{chunk, 174, 185...|
We conducted a ph...|[{document, 0, 14...|[{document, 0, 24...|[{token, 0, 1, We...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 70, 81, ...|[{named_entity, 0...|[{chunk, 70, 81, ...|
"""Monomeric sarc...|[{document, 0, 14...|[{document, 0, 14...|[{token, 0, 2, ""...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 138, 144...|[{named_entity, 0...|[{chunk, 0, 36, "...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 10 rows

In [0]:
posology_result.printSchema()

root
-- text: string (nullable = true)
-- document: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- sentence: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- token: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- embeddings: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- ner: array (nullable = true)
 |-- element: struct (containsNull = false)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- ner_chunk: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- ner_greedy: array (nullable = true)
 |-- element: struct (containsNull = false)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- ner_chunk_greedy: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)

In [0]:
from pyspark.sql.functions import monotonically_increasing_id

# This will return a new DF with all the columns + id
posology_result = posology_result.withColumn("id", monotonically_increasing_id())

posology_result.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---+
 text| document| sentence| token| embeddings| ner| ner_chunk| ner_greedy| ner_chunk_greedy| id|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---+
The human KCNJ9 (...|[{document, 0, 95...|[{document, 0, 12...|[{token, 0, 2, Th...|[{word_embeddings...|[{named_entity, 0...| []|[{named_entity, 0...|[{chunk, 52, 122,...| 0|
BACKGROUND: At pr...|[{document, 0, 14...|[{document, 0, 19...|[{token, 0, 9, BA...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 167, 180...|[{named_entity, 0...|[{chunk, 167, 180...| 1|
OBJECTIVE: To inv...|[{document, 0, 15...|[{document, 0, 14...|[{token, 0, 8, OB...|[{word_embeddings...|[{named_entity, 0...| []|[{named_entity, 0...| []| 2|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---+
only showing top 3 rows

In [0]:
posology_result.select('token.result','ner.result').show(truncate=100)

+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
 result| result|
+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
[The, human, KCNJ9, (, Kir, 3.3, ,, GIRK3, ), is, a, member, of, the, G-protein-activated, inward...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[BACKGROUND, :, At, present, ,, it, is, one, of, the, most, important, issues, for, the, treatmen...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-DRUG, O...|
[OBJECTIVE, :, To, investigate, the, relationship, between, preoperative, atrialfibrillation, and...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[Combined, EEG/fMRI, recording, has, been, used, to, localize, the, generators, of, EEGevents, an...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[Kohlschutter, syndrome, is, a, rare, neurodegenerative, disorder, presenting, withintractable, s...| [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]|
[Statistical, analysis, of, neuroimages, is, commonly, approached, with, intergroupcomparisons, m...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[The, synthetic, DOX-LNA, conjugate, was, characterized, by, proton, nuclear, magneticresonance, ...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[Our, objective, was, to, compare, three, different, methods, of, blood, pressuremeasurement, thr...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-DRUG, O, B-DRUG, O, O,...|
[We, conducted, a, phase, II, study, to, assess, the, efficacy, and, tolerability, ofirinotecan, ...|[O, O, O, O, O, O, O, O, O, O, O, O, B-DRUG, O, B-DRUG, O, O, B-DRUG, O, O, O, O, O, O, O, O, O, ...|
[""", Monomeric, sarcosine, oxidase, (, MSOX, ), is, a, flavoenzyme, that, catalyzes, the, oxidat...| [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-DRUG]|
[We, presented, the, tachinid, fly, Exorista, japonica, with, moving, host, models, :, afreeze-dr...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[The, literature, dealing, with, the, water, conducting, properties, of, sapwood, xylem, intrees,...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[A, novel, approach, to, synthesize, chitosan-O-isopropyl-5'-O-d4T, monophosphateconjugate, was, ...|[O, O, O, O, O, O, O, O, O, O, B-DRUG, I-DRUG, I-DRUG, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[An, HPLC-ESI-MS-MS, method, has, been, developed, for, the, quantitative, determination, offour,...|[O, O, O, O, O, O, O, O, O, O, O, O, O, B-DRUG, I-DRUG, I-DRUG, I-DRUG, O, O, O, O, O, O, O, O, O...|
[The, localizing, and, lateralizing, values, of, eye, and, head, ictal, deviations, duringfrontal...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[OBJECTIVE, :, To, evaluate, the, effectiveness, and, acceptability, of, expectantmanagement, of,...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[For, the, construction, of, new, combinatorial, libraries, ,, a, lead, compound, was, created, b...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[We, report, the, results, of, a, screen, for, genetic, association, with, urinary, arsenicmetabo...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,

In [0]:
from pyspark.sql import functions as F

posology_result_df = posology_result.select(F.explode(F.arrays_zip('token.result', 'ner.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("ner_label"))\
        .filter("ner_label!='O'")

posology_result_df.show(20, truncate=100)

+----------------+-----------+
 token| ner_label|
+----------------+-----------+
 anthracyclines| B-DRUG|
 taxanes| B-DRUG|
 vinorelbine| B-DRUG|
 vinorelbine| B-DRUG|
 anthracyclines| B-DRUG|
 taxanes| B-DRUG|
 Vinorelbinewas| B-DRUG|
 25| B-STRENGTH|
 mg/m(2| I-STRENGTH|
 intravenously| B-ROUTE|
 on|B-FREQUENCY|
 days|I-FREQUENCY|
tumorprogression| B-DRUG|
 thatvinorelbine| B-DRUG|
 anthracyclines| B-DRUG|
 taxanes| B-DRUG|
 tetrazolium| B-DRUG|
 dye| I-DRUG|
 DOX| B-DRUG|
 trandolapril| B-DRUG|
+----------------+-----------+
only showing top 20 rows

In [0]:
posology_greedy_result_df = posology_result.select(F.explode(F.arrays_zip('token.result', 'ner_greedy.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("ner_label"))\
        .filter("ner_label!='O'")

posology_greedy_result_df.show(20, truncate=100)

+-------------------+-----------+
 token| ner_label|
+-------------------+-----------+
G-protein-activated| B-DRUG|
 inwardly| I-DRUG|
 rectifying| I-DRUG|
 potassium| I-DRUG|
 (| I-DRUG|
 GIRK| I-DRUG|
 )| I-DRUG|
 channel| I-DRUG|
 family| I-DRUG|
 anthracyclines| B-DRUG|
 taxanes| B-DRUG|
 usefulnessof| B-DRUG|
 vinorelbine| I-DRUG|
 vinorelbine| B-DRUG|
 anthracyclines| B-DRUG|
 taxanes| B-DRUG|
 25| B-STRENGTH|
 mg/m(2| I-STRENGTH|
 intravenously| B-ROUTE|
 on|B-FREQUENCY|
+-------------------+-----------+
only showing top 20 rows

In [0]:
posology_result.select('id',F.explode(F.arrays_zip('ner_chunk.result', 'ner_chunk.begin', 'ner_chunk.end', 'ner_chunk.metadata')).alias("cols")) \
.select('id', F.expr("cols['3']['sentence']").alias("sentence_id"),
        F.expr("cols['0']").alias("chunk"),
        F.expr("cols['1']").alias("begin"),
        F.expr("cols['2']").alias("end"),
        F.expr("cols['3']['entity']").alias("ner_label"))\
        .filter("ner_label!='O'")\
        .show(truncate=False)

+---+-----------+----------------+-----+----+---------+
id |sentence_id|chunk |begin|end |ner_label|
+---+-----------+----------------+-----+----+---------+
1 |0 |anthracyclines |167 |180 |DRUG |
1 |0 |taxanes |186 |192 |DRUG |
1 |1 |vinorelbine |246 |256 |DRUG |
1 |1 |vinorelbine |386 |396 |DRUG |
1 |1 |anthracyclines |433 |446 |DRUG |
1 |1 |taxanes |452 |458 |DRUG |
1 |2 |Vinorelbinewas |470 |483 |DRUG |
1 |2 |25 mg/m(2 |517 |525 |STRENGTH |
1 |2 |intravenously |528 |540 |ROUTE |
1 |2 |on days |542 |548 |FREQUENCY|
1 |3 |tumorprogression|635 |650 |DRUG |
1 |11 |thatvinorelbine |1263 |1277|DRUG |
1 |11 |anthracyclines |1380 |1393|DRUG |
1 |11 |taxanes |1399 |1405|DRUG |
6 |2 |tetrazolium dye |432 |446 |DRUG |
6 |3 |DOX |576 |578 |DRUG |
7 |0 |trandolapril |174 |185 |DRUG |
7 |0 |losartan |191 |198 |DRUG |
7 |2 |3-week |313 |318 |DURATION |
7 |2 |placebo |320 |326 |DRUG |
+---+-----------+----------------+-----+----+---------+
only showing top 20 rows

In [0]:
posology_result.select('id',F.explode(F.arrays_zip('ner_chunk_greedy.result', 'ner_chunk_greedy.begin', 'ner_chunk_greedy.end', 'ner_chunk_greedy.metadata')).alias("cols")) \
.select('id', F.expr("cols['3']['sentence']").alias("sentence_id"),
        F.expr("cols['0']").alias("chunk"),
        F.expr("cols['1']").alias("begin"),
        F.expr("cols['2']").alias("end"),
        F.expr("cols['3']['entity']").alias("ner_label"))\
        .filter("ner_label!='O'")\
        .show(truncate=False)

+---+-----------+-----------------------------------------------------------------------+-----+----+---------+
id |sentence_id|chunk |begin|end |ner_label|
+---+-----------+-----------------------------------------------------------------------+-----+----+---------+
0 |0 |G-protein-activated inwardly rectifying potassium (GIRK) channel family|52 |122 |DRUG |
1 |0 |anthracyclines |167 |180 |DRUG |
1 |0 |taxanes |186 |192 |DRUG |
1 |1 |usefulnessof vinorelbine |233 |256 |DRUG |
1 |1 |vinorelbine |386 |396 |DRUG |
1 |1 |anthracyclines |433 |446 |DRUG |
1 |1 |taxanes |452 |458 |DRUG |
1 |2 |25 mg/m(2 |517 |525 |STRENGTH |
1 |2 |intravenously |528 |540 |ROUTE |
1 |2 |on days 1 and 8 of a3 week |542 |567 |FREQUENCY|
1 |5 |responserate |710 |721 |DRUG |
1 |10 |Themajor |1110 |1117|DRUG |
1 |11 |thatvinorelbine |1263 |1277|DRUG |
1 |11 |anthracyclines |1380 |1393|DRUG |
1 |11 |taxanes |1399 |1405|DRUG |
6 |2 |tetrazolium dye |432 |446 |DRUG |
6 |3 |DOX |576 |578 |DRUG |
7 |0 |trandolapril |174 |185 |DRUG |
7 |0 |losartan |191 |198 |DRUG |
7 |2 |placebo |320 |326 |DRUG |
+---+-----------+-----------------------------------------------------------------------+-----+----+---------+
only showing top 20 rows

In [0]:
posology_result.select('ner_chunk').take(2)[1][0][0].result

Out[39]: 'anthracyclines'

In [0]:
posology_result.select('ner_chunk').take(2)[1][0][0].metadata

Out[40]: {'sentence': '0', 'chunk': '0', 'entity': 'DRUG'}

### with LightPipelines

In [0]:
light_model = LightPipeline(posology_model)

text ='The patient was prescribed 1 capsule of Advil for 5 days . He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely for 3 months .'

light_result = light_model.annotate(text)

list(zip(light_result['token'], light_result['ner']))

Out[41]: [('The', 'O'),
 ('patient', 'O'),
 ('was', 'O'),
 ('prescribed', 'O'),
 ('1', 'B-DOSAGE'),
 ('capsule', 'B-FORM'),
 ('of', 'O'),
 ('Advil', 'B-DRUG'),
 ('for', 'B-DURATION'),
 ('5', 'I-DURATION'),
 ('days', 'I-DURATION'),
 ('.', 'O'),
 ('He', 'O'),
 ('was', 'O'),
 ('seen', 'O'),
 ('by', 'O'),
 ('the', 'O'),
 ('endocrinology', 'O'),
 ('service', 'O'),
 ('and', 'O'),
 ('she', 'O'),
 ('was', 'O'),
 ('discharged', 'O'),
 ('on', 'O'),
 ('40', 'B-DOSAGE'),
 ('units', 'I-DOSAGE'),
 ('of', 'O'),
 ('insulin', 'B-DRUG'),
 ('glargine', 'I-DRUG'),
 ('at', 'B-FREQUENCY'),
 ('night', 'I-FREQUENCY'),
 (',', 'O'),
 ('12', 'B-DOSAGE'),
 ('units', 'I-DOSAGE'),
 ('of', 'O'),
 ('insulin', 'B-DRUG'),
 ('lispro', 'I-DRUG'),
 ('with', 'B-FREQUENCY'),
 ('meals', 'I-FREQUENCY'),
 (',', 'O'),
 ('and', 'O'),
 ('metformin', 'B-DRUG'),
 ('1000', 'B-STRENGTH'),
 ('mg', 'I-STRENGTH'),
 ('two', 'B-FREQUENCY'),
 ('times', 'I-FREQUENCY'),
 ('a', 'I-FREQUENCY'),
 ('day', 'I-FREQUENCY'),
 ('.', 'O'),
 ('It', 'O'),
 ('was', 'O'),
 ('determined', 'O'),
 ('that', 'O'),
 ('all', 'O'),
 ('SGLT2', 'B-DRUG'),
 ('inhibitors', 'I-DRUG'),
 ('should', 'O'),
 ('be', 'O'),
 ('discontinued', 'O'),
 ('indefinitely', 'O'),
 ('for', 'B-DURATION'),
 ('3', 'I-DURATION'),
 ('months', 'I-DURATION'),
 ('.', 'O')]

In [0]:
list(zip(light_result['token'], light_result['ner_greedy']))

Out[42]: [('The', 'O'),
 ('patient', 'O'),
 ('was', 'O'),
 ('prescribed', 'O'),
 ('1', 'B-DRUG'),
 ('capsule', 'I-DRUG'),
 ('of', 'I-DRUG'),
 ('Advil', 'I-DRUG'),
 ('for', 'B-DURATION'),
 ('5', 'I-DURATION'),
 ('days', 'I-DURATION'),
 ('.', 'O'),
 ('He', 'O'),
 ('was', 'O'),
 ('seen', 'O'),
 ('by', 'O'),
 ('the', 'O'),
 ('endocrinology', 'O'),
 ('service', 'O'),
 ('and', 'O'),
 ('she', 'O'),
 ('was', 'O'),
 ('discharged', 'O'),
 ('on', 'O'),
 ('40', 'B-DRUG'),
 ('units', 'I-DRUG'),
 ('of', 'I-DRUG'),
 ('insulin', 'I-DRUG'),
 ('glargine', 'I-DRUG'),
 ('at', 'B-FREQUENCY'),
 ('night', 'I-FREQUENCY'),
 (',', 'O'),
 ('12', 'B-DRUG'),
 ('units', 'I-DRUG'),
 ('of', 'I-DRUG'),
 ('insulin', 'I-DRUG'),
 ('lispro', 'I-DRUG'),
 ('with', 'B-FREQUENCY'),
 ('meals', 'I-FREQUENCY'),
 (',', 'O'),
 ('and', 'O'),
 ('metformin', 'B-DRUG'),
 ('1000', 'I-DRUG'),
 ('mg', 'I-DRUG'),
 ('two', 'B-FREQUENCY'),
 ('times', 'I-FREQUENCY'),
 ('a', 'I-FREQUENCY'),
 ('day', 'I-FREQUENCY'),
 ('.', 'O'),
 ('It', 'O'),
 ('was', 'O'),
 ('determined', 'O'),
 ('that', 'O'),
 ('all', 'O'),
 ('SGLT2', 'B-DRUG'),
 ('inhibitors', 'I-DRUG'),
 ('should', 'O'),
 ('be', 'O'),
 ('discontinued', 'O'),
 ('indefinitely', 'O'),
 ('for', 'B-DURATION'),
 ('3', 'I-DURATION'),
 ('months', 'I-DURATION'),
 ('.', 'O')]

In [0]:
light_result = light_model.fullAnnotate(text)

chunks = []
entities = []
begin =[]
end = []

for n in light_result[0]['ner_chunk']:
        
    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    
import pandas as pd

df = pd.DataFrame({'chunks':chunks, 'entities':entities,
                   'begin': begin, 'end': end})

df

Out[43]:

,chunks,entities,begin,end
0,1,DOSAGE,27,27
1,capsule,FORM,29,35
2,Advil,DRUG,40,44
3,for 5 days,DURATION,46,55
4,40 units,DOSAGE,126,133
5,insulin glargine,DRUG,138,153
6,at night,FREQUENCY,155,162
7,12 units,DOSAGE,166,173
8,insulin lispro,DRUG,178,191
9,with meals,FREQUENCY,193,202


#### NER Visualization

In [0]:
from sparknlp_display import NerVisualizer

visualiser = NerVisualizer()

ner_vis = visualiser.display(light_result[0], label_col='ner_chunk', document_col='document', return_html=True)
  
displayHTML(ner_vis)

The patient was prescribed 1 DOSAGE capsule FORM of Advil DRUG for 5 days DURATION . He was seen by the endocrinology service and she was discharged on 40 units DOSAGE of insulin glargine DRUG at night FREQUENCY , 12 units DOSAGE of insulin lispro DRUG with meals FREQUENCY , and metformin DRUG 1000 mg STRENGTH two times a day FREQUENCY . It was determined that all SGLT2 inhibitors DRUG should be discontinued indefinitely for 3 months DURATION .

In [0]:
# ner_greedy

visualiser_greedy = NerVisualizer()

ner_greedy_vis = visualiser_greedy.display(light_result[0], label_col='ner_chunk_greedy', document_col='document', return_html=True)

displayHTML(ner_greedy_vis)

The patient was prescribed 1 capsule of Advil DRUG for 5 days DURATION . He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine DRUG at night FREQUENCY , 12 units of insulin lispro DRUG with meals FREQUENCY , and metformin 1000 mg DRUG two times a day FREQUENCY . It was determined that all SGLT2 inhibitors DRUG should be discontinued indefinitely for 3 months DURATION .

## Writing a generic NER function

In [0]:
def get_base_pipeline (embeddings = 'embeddings_clinical'):

  documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

  # Sentence Detector annotator, processes various sentences per line

  sentenceDetector = SentenceDetector()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")

  # Tokenizer splits words in a relevant format for NLP

  tokenizer = Tokenizer()\
      .setInputCols(["sentence"])\
      .setOutputCol("token")

  # Clinical word embeddings trained on PubMED dataset
  word_embeddings = WordEmbeddingsModel.pretrained(embeddings, "en", "clinical/models")\
      .setInputCols(["sentence", "token"])\
      .setOutputCol("embeddings")

  base_pipeline = Pipeline(stages=[
                    documentAssembler,
                    sentenceDetector,
                    tokenizer,
                    word_embeddings
                  ])

  return base_pipeline



def get_clinical_entities (embeddings, spark_df, nrows = 100, model_name = 'ner_clinical'):

  # NER model trained on i2b2 (sampled from MIMIC) dataset
  loaded_ner_model = MedicalNerModel.pretrained(model_name, "en", "clinical/models") \
      .setInputCols(["sentence", "token", "embeddings"]) \
      .setOutputCol("ner")

  ner_converter = NerConverter() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")

  base_pipeline = get_base_pipeline (embeddings)

  nlpPipeline = Pipeline(stages=[
      base_pipeline,
      loaded_ner_model,
      ner_converter])

  empty_data = spark.createDataFrame([[""]]).toDF("text")

  model = nlpPipeline.fit(empty_data)

  result = model.transform(spark_df.limit(nrows))

  result = result.withColumn("id", monotonically_increasing_id())

  result_df = result.select('id',F.explode(F.arrays_zip('ner_chunk.result', 'ner_chunk.begin', 'ner_chunk.end', 'ner_chunk.metadata')).alias("cols")) \
  .select('id', F.expr("cols['3']['sentence']").alias("sentence_id"),
          F.expr("cols['0']").alias("chunk"),
          F.expr("cols['1']").alias("begin"),
          F.expr("cols['2']").alias("end"),
          F.expr("cols['3']['entity']").alias("ner_label"))\
          .filter("ner_label!='O'")

  return result_df

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_clinical'

nrows = 100

ner_df = get_clinical_entities (embeddings, pubMedDF, nrows, model_name)

ner_df.show()

ner_clinical download started this may take some time.
Approximate size to download 13.9 MB
[ | ][ / ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
+---+-----------+--------------------+-----+---+---------+
 id|sentence_id| chunk|begin|end|ner_label|
+---+-----------+--------------------+-----+---+---------+
 0| 0|the G-protein-act...| 48|106|TREATMENT|
 0| 1|the genomicorgani...| 142|164|TREATMENT|
 0| 1|a candidate gene ...| 210|254| PROBLEM|
 0| 2| byapproximately| 380|394|TREATMENT|
 0| 3|single nucleotide...| 464|494|TREATMENT|
 0| 3|aVal366Ala substi...| 532|554|TREATMENT|
 0| 3| an 8 base-pair| 561|574|TREATMENT|
 0| 4| insertion/deletion| 581|598| PROBLEM|
 0| 5|Ourexpression stu...| 601|621| TEST|
 0| 5|the transcript in...| 648|685| PROBLEM|
 0| 5|fat andskeletal m...| 749|770| PROBLEM|
 0| 6| furtherstudies| 830|843| PROBLEM|
 0| 6| the KCNJ9 protein| 864|880|TREATMENT|
 0| 6| evaluation| 892|901| TEST|
 0| 6| Type II diabetes| 940|955| PROBLEM|
 1| 0| the treatment| 67| 79|TREATMENT|
 1| 0| breast cancer| 84| 96| PROBLEM|
 1| 0|the standard therapy| 109|128|TREATMENT|
 1| 0| anthracyclines| 167|180|TREATMENT|
 1| 0| taxanes| 186|192|TREATMENT|
+---+-----------+--------------------+-----+---+---------+
only showing top 20 rows

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_posology'

nrows = 100

ner_df = get_clinical_entities (embeddings, pubMedDF, nrows, model_name)

ner_df.show()

ner_posology download started this may take some time.
Approximate size to download 13.8 MB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
+---+-----------+----------------+-----+----+---------+
 id|sentence_id| chunk|begin| end|ner_label|
+---+-----------+----------------+-----+----+---------+
 1| 0| anthracyclines| 167| 180| DRUG|
 1| 0| taxanes| 186| 192| DRUG|
 1| 1| vinorelbine| 246| 256| DRUG|
 1| 1| vinorelbine| 386| 396| DRUG|
 1| 1| anthracyclines| 433| 446| DRUG|
 1| 1| taxanes| 452| 458| DRUG|
 1| 2| Vinorelbinewas| 470| 483| DRUG|
 1| 2| 25 mg/m(2| 517| 525| STRENGTH|
 1| 2| intravenously| 528| 540| ROUTE|
 1| 2| on days| 542| 548|FREQUENCY|
 1| 3|tumorprogression| 635| 650| DRUG|
 1| 10| thatvinorelbine| 1263|1277| DRUG|
 1| 10| anthracyclines| 1380|1393| DRUG|
 1| 10| taxanes| 1399|1405| DRUG|
 6| 2| tetrazolium dye| 432| 446| DRUG|
 6| 3| DOX| 576| 578| DRUG|
 7| 0| trandolapril| 174| 185| DRUG|
 7| 0| losartan| 191| 198| DRUG|
 7| 2| 3-week| 313| 318| DURATION|
 7| 2| placebo| 320| 326| DRUG|
+---+-----------+----------------+-----+----+---------+
only showing top 20 rows

In [0]:
import pandas as pd

def get_clinical_entities_light (light_model, text):

  light_result = light_model.fullAnnotate(text)

  chunks = []
  entities = []

  for n in light_result[0]['ner_chunk']:
          
      chunks.append(n.result)
      entities.append(n.metadata['entity']) 
      
  df = pd.DataFrame({'chunks':chunks, 'entities':entities})

  return df

In [0]:
text ='The patient was prescribed 1 capsule of Parol with meals . He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely fro 3 months .'

light_model = LightPipeline(posology_model)

get_clinical_entities_light (light_model, text)

Out[50]:

,chunks,entities
0,1,DOSAGE
1,capsule,FORM
2,Parol,DRUG
3,with meals,FREQUENCY
4,40 units,DOSAGE
5,insulin glargine,DRUG
6,at night,FREQUENCY
7,12 units,DOSAGE
8,insulin lispro,DRUG
9,with meals,FREQUENCY


## PHI NER

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_deid_large'

# deidentify_dl
# ner_deid_large

nrows = 100

ner_df = get_clinical_entities (embeddings, pubMedDF, nrows, model_name)

pd_ner_df = ner_df.toPandas()

ner_deid_large download started this may take some time.
Approximate size to download 14.1 MB
[ | ][ / ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]

In [0]:
pd_ner_df.sample(20)

Out[52]:

,id,sentence_id,chunk,begin,end,ner_label
50,57,4,Korea,646,650,LOCATION
22,27,0,Malaysia,172,179,LOCATION
64,72,2,Hurricane Katrina,409,425,NAME
37,44,3,January1st,458,467,LOCATION
72,77,9,trucking company,1567,1582,LOCATION
16,24,7,Japan,1263,1267,LOCATION
63,71,2,April 2005,482,491,DATE
39,44,3,1950,494,497,LOCATION
1,2,0,2005,280,283,DATE
54,57,15,Korea,1884,1888,LOCATION


In [0]:
pd_ner_df.ner_label.value_counts()

Out[53]: LOCATION 54
DATE 15
NAME 7
PROFESSION 4
AGE 3
Name: ner_label, dtype: int64

## BioNLP (Cancer Genetics) NER

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_bionlp'

nrows = 100

ner_df = get_clinical_entities (embeddings, pubMedDF, nrows, model_name)

ner_df.show(truncate = False)

ner_bionlp download started this may take some time.
Approximate size to download 13.9 MB
[ | ][ / ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
+---+-----------+----------------------+-----+---+--------------------+
id |sentence_id|chunk |begin|end|ner_label |
+---+-----------+----------------------+-----+---+--------------------+
0 |0 |human |4 |8 |Organism |
0 |0 |Kir 3.3 |17 |23 |Gene_or_gene_product|
0 |0 |GIRK3 |26 |30 |Gene_or_gene_product|
0 |0 |potassium |92 |100|Simple_chemical |
0 |0 |GIRK |103 |106|Gene_or_gene_product|
0 |1 |chromosome 1q21-23 |188 |205|Cellular_component |
0 |5 |pancreas |697 |704|Organ |
0 |5 |tissues |740 |746|Tissue |
0 |5 |fat andskeletal muscle|749 |770|Tissue |
0 |6 |KCNJ9 |801 |805|Gene_or_gene_product|
0 |6 |Type II |940 |946|Gene_or_gene_product|
1 |0 |breast cancer |84 |96 |Cancer |
1 |0 |patients |134 |141|Organism |
1 |0 |anthracyclines |167 |180|Simple_chemical |
1 |0 |taxanes |186 |192|Simple_chemical |
1 |1 |vinorelbine |246 |256|Simple_chemical |
1 |1 |patients |273 |280|Organism |
1 |1 |breast |309 |314|Cancer |
1 |1 |vinorelbine inpatients|386 |407|Simple_chemical |
1 |1 |anthracyclines |433 |446|Simple_chemical |
+---+-----------+----------------------+-----+---+--------------------+
only showing top 20 rows

# NER Chunker
We can extract phrases that fits into a known pattern using the NER tags. NerChunker would be quite handy to extract entity groups with neighboring tokens when there is no pretrained NER model to address certain issues. Lets say we want to extract clinical findings and body parts together as a single chunk even if there are some unwanted tokens between.

In [0]:
posology_ner = MedicalNerModel.pretrained("ner_posology", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")

ner_chunker = NerChunker()\
    .setInputCols(["sentence","ner"])\
    .setOutputCol("ner_chunk")\
    .setRegexParsers(["<DRUG>.*<FREQUENCY>"])

nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    posology_ner,
    ner_chunker])

empty_data = spark.createDataFrame([[""]]).toDF("text")

ner_chunker_model = nlpPipeline.fit(empty_data)

ner_posology download started this may take some time.
Approximate size to download 13.8 MB
[ | ][OK!]

In [0]:
posology_ner.getClasses()

Out[56]: ['O',
 'B-DOSAGE',
 'B-STRENGTH',
 'I-STRENGTH',
 'B-ROUTE',
 'B-FREQUENCY',
 'I-FREQUENCY',
 'B-DRUG',
 'I-DRUG',
 'B-FORM',
 'I-DOSAGE',
 'B-DURATION',
 'I-DURATION',
 'I-FORM',
 'I-ROUTE']

In [0]:
light_model = LightPipeline(ner_chunker_model)

text ='The patient was prescribed 1 capsule of Advil for 5 days . He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely fro 3 months .'

light_result = light_model.annotate(text)

list(zip(light_result['token'], light_result['ner']))

Out[57]: [('The', 'O'),
 ('patient', 'O'),
 ('was', 'O'),
 ('prescribed', 'O'),
 ('1', 'B-DOSAGE'),
 ('capsule', 'B-FORM'),
 ('of', 'O'),
 ('Advil', 'B-DRUG'),
 ('for', 'B-DURATION'),
 ('5', 'I-DURATION'),
 ('days', 'I-DURATION'),
 ('.', 'O'),
 ('He', 'O'),
 ('was', 'O'),
 ('seen', 'O'),
 ('by', 'O'),
 ('the', 'O'),
 ('endocrinology', 'O'),
 ('service', 'O'),
 ('and', 'O'),
 ('she', 'O'),
 ('was', 'O'),
 ('discharged', 'O'),
 ('on', 'O'),
 ('40', 'B-DOSAGE'),
 ('units', 'I-DOSAGE'),
 ('of', 'O'),
 ('insulin', 'B-DRUG'),
 ('glargine', 'I-DRUG'),
 ('at', 'B-FREQUENCY'),
 ('night', 'I-FREQUENCY'),
 (',', 'O'),
 ('12', 'B-DOSAGE'),
 ('units', 'I-DOSAGE'),
 ('of', 'O'),
 ('insulin', 'B-DRUG'),
 ('lispro', 'I-DRUG'),
 ('with', 'B-FREQUENCY'),
 ('meals', 'I-FREQUENCY'),
 (',', 'O'),
 ('metformin', 'B-DRUG'),
 ('1000', 'B-STRENGTH'),
 ('mg', 'I-STRENGTH'),
 ('two', 'B-FREQUENCY'),
 ('times', 'I-FREQUENCY'),
 ('a', 'I-FREQUENCY'),
 ('day', 'I-FREQUENCY'),
 ('.', 'O'),
 ('It', 'O'),
 ('was', 'O'),
 ('determined', 'O'),
 ('that', 'O'),
 ('all', 'O'),
 ('SGLT2', 'B-DRUG'),
 ('inhibitors', 'I-DRUG'),
 ('should', 'O'),
 ('be', 'O'),
 ('discontinued', 'O'),
 ('indefinitely', 'O'),
 ('fro', 'O'),
 ('3', 'O'),
 ('months', 'O'),
 ('.', 'O')]

In [0]:
light_result["ner_chunk"]

Out[58]: ['insulin glargine at night , 12 units of insulin lispro with meals , metformin 1000 mg two times a day']

#Chunk Filterer
ChunkFilterer will allow you to filter out named entities by some conditions or predefined look-up lists, so that you can feed these entities to other annotators like Assertion Status or Entity Resolvers. It can be used with two criteria: isin and regex.

In [0]:
posology_ner = MedicalNerModel.pretrained("ner_posology", "en", "clinical/models") \
      .setInputCols(["sentence", "token", "embeddings"]) \
      .setOutputCol("ner")

ner_converter = NerConverter()\
      .setInputCols(["sentence","token","ner"])\
      .setOutputCol("ner_chunk")
      
chunk_filterer = ChunkFilterer()\
      .setInputCols("sentence","ner_chunk")\
      .setOutputCol("chunk_filtered")\
      .setCriteria("isin")\
      .setWhiteList(['Advil','metformin', 'insulin lispro'])

nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    posology_ner,
    ner_converter,
    chunk_filterer])

empty_data = spark.createDataFrame([[""]]).toDF("text")

chunk_filter_model = nlpPipeline.fit(empty_data)

ner_posology download started this may take some time.
Approximate size to download 13.8 MB
[ | ][OK!]

In [0]:
light_model = LightPipeline(chunk_filter_model)

text ='The patient was prescribed 1 capsule of Advil for 5 days . He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely fro 3 months .'

light_result = light_model.annotate(text)

light_result.keys()

Out[60]: dict_keys(['document', 'ner_chunk', 'chunk_filtered', 'token', 'ner', 'embeddings', 'sentence'])

In [0]:
light_result['ner_chunk'] 

Out[61]: ['1',
 'capsule',
 'Advil',
 'for 5 days',
 '40 units',
 'insulin glargine',
 'at night',
 '12 units',
 'insulin lispro',
 'with meals',
 'metformin',
 '1000 mg',
 'two times a day',
 'SGLT2 inhibitors']

In [0]:
light_result["chunk_filtered"]

Out[62]: ['Advil', 'insulin lispro', 'metformin']

In [0]:
ner_model = MedicalNerModel.pretrained("ner_clinical", "en", "clinical/models")\
      .setInputCols("sentence","token","embeddings")\
      .setOutputCol("ner")
      
chunk_filterer = ChunkFilterer()\
      .setInputCols("sentence","ner_chunk")\
      .setOutputCol("chunk_filtered")\
      .setCriteria("isin")\
      .setWhiteList(['severe fever','sore throat'])

nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    ner_model,
    ner_converter,
    chunk_filterer])

empty_data = spark.createDataFrame([[""]]).toDF("text")

chunk_filter_model = nlpPipeline.fit(empty_data)

ner_clinical download started this may take some time.
Approximate size to download 13.9 MB
[ | ][OK!]

In [0]:
text = 'Patient with severe fever, severe cough, sore throat, stomach pain, and a headache.'

filter_df = spark.createDataFrame([[text]]).toDF('text')

chunk_filter_result = chunk_filter_model.transform(filter_df)

In [0]:
chunk_filter_result.select('ner_chunk.result', 'chunk_filtered.result').show(truncate=False)

+-------------------------------------------------------------------+---------------------------+
result |result |
+-------------------------------------------------------------------+---------------------------+
[severe fever, severe cough, sore throat, stomach pain, a headache]|[severe fever, sore throat]|
+-------------------------------------------------------------------+---------------------------+

## Training a Clinical NER (NCBI Disease Dataset)

In [0]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/NER_NCBIconlltrain.txt
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/NER_NCBIconlltest.txt
  
dbutils.fs.cp("file:/databricks/driver/NER_NCBIconlltest.txt", "dbfs:/")
dbutils.fs.cp("file:/databricks/driver/NER_NCBIconlltrain.txt", "dbfs:/")

Out[8]: True

In [0]:
from sparknlp.training import CoNLL

conll_data = CoNLL().readDataset(spark, 'file:/databricks/driver/NER_NCBIconlltrain.txt')

conll_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| pos| label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
Identification of...|[{document, 0, 89...|[{document, 0, 89...|[{token, 0, 13, I...|[{pos, 0, 13, NN,...|[{named_entity, 0...|
The adenomatous p...|[{document, 0, 21...|[{document, 0, 21...|[{token, 0, 2, Th...|[{pos, 0, 2, NN, ...|[{named_entity, 0...|
Complex formation...|[{document, 0, 63...|[{document, 0, 63...|[{token, 0, 6, Co...|[{pos, 0, 6, NN, ...|[{named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows

In [0]:
conll_data.count()

Out[39]: 3266

In [0]:
from pyspark.sql import functions as F

conll_data.select(F.explode(F.arrays_zip('token.result','label.result')).alias("cols")) \
          .select(F.expr("cols['0']").alias("token"),
                  F.expr("cols['1']").alias("ground_truth")).groupBy('ground_truth').count().orderBy('count', ascending=False).show(100,truncate=False)


+------------+-----+
ground_truth|count|
+------------+-----+
O |75093|
I-Disease |3547 |
B-Disease |3093 |
+------------+-----+

In [0]:
conll_data.select("label.result").distinct().count()

Out[41]: 1537

In [0]:
import pyspark.sql.functions as F

#conll_data.select(F.countDistinct("label.result")).show()
#conll_data.groupBy("label.result").count().show(truncate=False)

conll_data = conll_data.withColumn('unique', F.array_distinct("label.result"))\
                       .withColumn('c', F.size('unique'))\
                       .filter(F.col('c')>1)

conll_data.select(F.explode(F.arrays_zip('token.result','label.result')).alias("cols")) \
          .select(F.expr("cols['0']").alias("token"),
                  F.expr("cols['1']").alias("ground_truth")).groupBy('ground_truth').count().orderBy('count', ascending=False).show(100,truncate=False)



+------------+-----+
ground_truth|count|
+------------+-----+
O |39427|
I-Disease |3547 |
B-Disease |3093 |
+------------+-----+

In [0]:
# Clinical word embeddings trained on PubMED dataset
clinical_embeddings = WordEmbeddingsModel.pretrained('embeddings_clinical', "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]

In [0]:
test_data = CoNLL().readDataset(spark, "file:/databricks/driver/NER_NCBIconlltest.txt")

test_data = clinical_embeddings.transform(test_data)

test_data.write.parquet('/NER_NCBIconlltest.parquet')

## MedicalNER Graph

In [0]:
!mkdir ner_logs
!mkdir medical_ner_graph

!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/nerdl_graph/blstm_12_200_128_83_1.pb -P /databricks/driver/medical_ner_graph

In [0]:
# for a custom MedicalNER graph

'''
!pip -q install tensorflow==1.15.0

from sparknlp_jsl.training import tf_graph
tf_graph.print_model_params("ner_dl")
tf_graph.build("ner_dl", build_params={"embeddings_dim": 200, "nchars": 83, "ntags": 12, "is_medical": 1}, model_location="/medical_ner_graphs", model_filename="auto")
'''

In [0]:
# for open source users
'''
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/create_graph.py
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/dataset_encoder.py
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/ner_model.py
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/ner_model_saver.py
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/sentence_grouper.py

!pip -q install tensorflow==1.15.0

import create_graph

ntags = 3 # number of labels
embeddings_dim = 200
nchars =83

create_graph.create_graph(ntags, embeddings_dim, nchars)
'''

In [0]:
nerTagger = MedicalNerApproach()\
  .setInputCols(["sentence", "token", "embeddings"])\
  .setLabelColumn("label")\
  .setOutputCol("ner")\
  .setMaxEpochs(2)\
  .setBatchSize(64)\
  .setRandomSeed(0)\
  .setVerbose(1)\
  .setValidationSplit(0.2)\
  .setEvaluationLogExtended(True) \
  .setEnableOutputLogs(True)\
  .setIncludeConfidence(True)\
  .setTestDataset("/NER_NCBIconlltest.parquet")\
  .setOutputLogsPath('file:/databricks/driver/ner_logs')\
  .setGraphFolder('file:/databricks/driver/medical_ner_graph/') # control your graph folder

ner_pipeline = Pipeline(stages=[
          clinical_embeddings,
          nerTagger
 ])

In [0]:
# 2 epochs 47 sec
ner_model = ner_pipeline.fit(conll_data)

# if you get an error for incompatible TF graph, use 4.1 NerDL-Graph.ipynb notebook to create a graph
# licensed users can also use 17.Graph_builder_for_DL_models.ipynb to create tf graphs easily.

In [0]:
%sh cd /databricks/driver/ner_logs && ls -lt

total 8
-rw-r--r-- 1 root root 1936 Jul 10 14:37 MedicalNerApproach_b03b6408c52f.log
-rw-r--r-- 1 root root 1972 Jul 10 14:29 MedicalNerApproach_4bc740dd09c9.log

In [0]:
%sh cat /databricks/driver/ner_logs/MedicalNerApproach_b03b6408c52f.log

Name of the selected graph: /databricks/driver/medical_ner_graph/blstm_12_200_128_83_1.pb
Training started - total epochs: 2 - lr: 0.001 - batch size: 64 - labels: 3 - chars: 80 - training examples: 1373


Epoch 1/2 started, lr: 0.001, dataset size: 1373


Epoch 1/2 - 7.54s - loss: 503.25476 - batches: 25
Quality on validation dataset (20.0%), validation examples = 274
time to finish evaluation: 2.55s
label tp fp fn prec rec f1
I-Disease 0 0 622 0.0 0.0 0.0
B-Disease 54 59 506 0.4778761 0.09642857 0.16047548
tp: 54 fp: 59 fn: 1128 labels: 2
Macro-average prec: 0.23893805, rec: 0.048214287, f1: 0.08023774
Micro-average prec: 0.4778761, rec: 0.04568528, f1: 0.08339769
Quality on test dataset: 
time to finish evaluation: 4.73s
label tp fp fn prec rec f1
I-Disease 0 0 789 0.0 0.0 0.0
B-Disease 72 66 636 0.5217391 0.10169491 0.17021276
tp: 72 fp: 66 fn: 1425 labels: 2
Macro-average prec: 0.26086956, rec: 0.050847456, f1: 0.08510638
Micro-average prec: 0.5217391, rec: 0.04809619, f1: 0.088073395


Epoch 2/2 started, lr: 9.950249E-4, dataset size: 1373


Epoch 2/2 - 6.37s - loss: 231.83836 - batches: 25
Quality on validation dataset (20.0%), validation examples = 274
time to finish evaluation: 2.27s
label tp fp fn prec rec f1
I-Disease 256 60 366 0.8101266 0.41157556 0.54584223
B-Disease 221 72 339 0.7542662 0.39464286 0.51817113
tp: 477 fp: 132 fn: 705 labels: 2
Macro-average prec: 0.7821964, rec: 0.4031092, f1: 0.53203255
Micro-average prec: 0.7832512, rec: 0.4035533, f1: 0.5326633
Quality on test dataset: 
time to finish evaluation: 4.72s
label tp fp fn prec rec f1
I-Disease 290 94 499 0.7552083 0.36755386 0.49445862
B-Disease 251 94 457 0.72753626 0.35451978 0.47673315
tp: 541 fp: 188 fn: 956 labels: 2
Macro-average prec: 0.7413723, rec: 0.36103684, f1: 0.48559594
Micro-average prec: 0.74211246, rec: 0.36138946, f1: 0.48607364

### Evaluate your model

In [0]:
pred_df = ner_model.stages[1].transform(test_data)

In [0]:
pred_df.columns

Out[87]: ['text', 'document', 'sentence', 'token', 'pos', 'label', 'embeddings', 'ner']

In [0]:
from sparknlp_jsl.eval import NerDLMetrics
import pyspark.sql.functions as F

evaler = NerDLMetrics(mode="full_chunk", dropO=True)

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label").cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
    .withColumn("recall", F.round(eval_result["recall"],4))\
    .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+-----+-----+-----+---------+------+------+
 entity| tp| fp| fn|total|precision|recall| f1|
+-------+-----+-----+-----+-----+---------+------+------+
Disease|230.0|305.0|474.0|704.0| 0.4299|0.3267|0.3713|
+-------+-----+-----+-----+-----+---------+------+------+

+-------------------+
 macro|
+-------------------+
0.37126715092816787|
+-------------------+

None
+-------------------+
 micro|
+-------------------+
0.37126715092816787|
+-------------------+

None

In [0]:
evaler = NerDLMetrics(mode="partial_chunk_per_token", dropO=True)

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label").cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
    .withColumn("recall", F.round(eval_result["recall"],4))\
    .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+----+-----+------+---------+------+------+
 entity| tp| fp| fn| total|precision|recall| f1|
+-------+-----+----+-----+------+---------+------+------+
Disease|662.0|67.0|835.0|1497.0| 0.9081|0.4422|0.5948|
+-------+-----+----+-----+------+---------+------+------+

+------------------+
 macro|
+------------------+
0.5947888589398024|
+------------------+

None
+------------------+
 micro|
+------------------+
0.5947888589398024|
+------------------+

None

In [0]:
ner_model.stages[1].write().overwrite().save('/databricks/driver/models/custom_Med_NER_2e')

In [0]:
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence = SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentence')

token = Tokenizer()\
    .setInputCols(['sentence'])\
    .setOutputCol('token')

loaded_ner_model = MedicalNerModel.load("dbfs:/databricks/driver/models/custom_Med_NER_2e")\
 .setInputCols(["sentence", "token", "embeddings"])\
 .setOutputCol("ner")

converter = NerConverter()\
  .setInputCols(["document", "token", "ner"])\
  .setOutputCol("ner_span")

ner_prediction_pipeline = Pipeline(
    stages = [
        document,
        sentence,
        token,
        clinical_embeddings,
        loaded_ner_model,
        converter])

empty_data = spark.createDataFrame([['']]).toDF("text")

prediction_model = ner_prediction_pipeline.fit(empty_data)

from sparknlp.base import LightPipeline

light_model = LightPipeline(prediction_model)

In [0]:
text = "She has a metastatic breast cancer to lung"

def get_preds(text, light_model):

    result = light_model.fullAnnotate(text)[0]

    return [(i.result, i.metadata['entity']) for i in result['ner_span']]

get_preds(text, light_model)

Out[92]: [('metastatic breast cancer', 'Disease')]

End of Notebook #